# Human Activity Recognition

In [2]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/XOEN9W05_4A"' 
     'frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

Database built from the recordings of 30 subjects performing activities of daily living while carrying a waist-mounted smartphone with embedded inertial sensors.

Each person performed six activities (WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING)

Sources:

Project: https://github.com/guillaume-chevalier/LSTM-Human-Activity-Recognition

Data: https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones

# Load Libraries

In [154]:
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from hdfs import InsecureClient
import pandas as pd
from datetime import datetime

# Connect to Hadoop

In [165]:
client_hdfs = InsecureClient('http://awscdh6-ma.sap.local:9870', user='dr.who')

In [166]:
client_hdfs.list('/tmp/tbr/BARMER/DSP')

['data_labeled_performance',
 'data_labeled_training',
 'data_unlabeled_predictions',
 'model']

# Load Data from Hadoop

In [117]:
# Check Trainings Data
client_hdfs.list('/tmp/tbr/BARMER/DSP/data_labeled_training/Inertial Signals')

['body_acc_x.txt',
 'body_acc_y.txt',
 'body_acc_z.txt',
 'body_gyro_x.txt',
 'body_gyro_y.txt',
 'body_gyro_z.txt',
 'total_acc_x.txt',
 'total_acc_y.txt',
 'total_acc_z.txt',
 'y_labels.txt']

Each axis of each signal is stored in a separate file, meaning that each of the train and test datasets have nine input files to load and one output file to load.

Function for loading the entire dataset:

In [118]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix):
    # load data and labels
    X, y = load_dataset_group(prefix)
    
    # zero-offset class values
    y = y - 1
    
    # one hot encode y
    y = to_categorical(y)
    
    # return dataset
    return X, y

A function for loading a dataset group of files

In [119]:
# load a dataset group, such as train or test
def load_dataset_group(group):
    
    # load all 9 files as a single array
    filenames = list()
    
    # total acceleration
    filenames += ['/Inertial Signals/total_acc_x.txt',
                  '/Inertial Signals/total_acc_y.txt',
                  '/Inertial Signals/total_acc_z.txt']
    
    # body acceleration
    filenames += ['/Inertial Signals/body_acc_x.txt',
                  '/Inertial Signals/body_acc_y.txt',
                  '/Inertial Signals/body_acc_z.txt']
    
    # body gyroscope
    filenames += ['/Inertial Signals/body_gyro_x.txt',
                  '/Inertial Signals/body_gyro_y.txt',
                  '/Inertial Signals/body_gyro_z.txt']
    
    # load input data
    X = load_group(filenames, group)
    
    # load class output
    y = load_file(group+'/Inertial Signals/y_labels.txt')
    
    # return X and y
    return X, y

A function for loading a group of files

In [120]:
# load a list of files and return as a 3d numpy array
def load_group(filenames, group):
    loaded = list()
    
    for name in filenames:

        data = load_file(group+name)
        loaded.append(data)
    
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded

A function for loading a single file

In [121]:
# load a single file as a numpy array
def load_file(filepath):
    #dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    
    path = '/tmp/tbr/BARMER/DSP/' + filepath
     
    with client_hdfs.read(path, encoding = 'utf-8') as reader:
        dataframe = read_csv(reader, header=None, delim_whitespace=True)
        
    return dataframe.values

Execute Function-Chain

In [122]:
# load training data
trainX, trainy = load_dataset('data_labeled_training')

In [123]:
# load test data
testX, testy = load_dataset('data_labeled_performance')

In [124]:
trainX[0]

array([[ 1.012817e+00, -1.232167e-01,  1.029341e-01, ...,  3.019122e-02,
         6.601362e-02,  2.285864e-02],
       [ 1.022833e+00, -1.268756e-01,  1.056872e-01, ...,  4.371071e-02,
         4.269897e-02,  1.031572e-02],
       [ 1.022028e+00, -1.240037e-01,  1.021025e-01, ...,  3.568780e-02,
         7.485018e-02,  1.324969e-02],
       ...,
       [ 1.018445e+00, -1.240696e-01,  1.003852e-01, ...,  3.985177e-02,
         1.909445e-03, -2.170124e-03],
       [ 1.019372e+00, -1.227451e-01,  9.987355e-02, ...,  3.744932e-02,
        -7.982483e-05, -5.642633e-03],
       [ 1.021171e+00, -1.213260e-01,  9.498741e-02, ...,  2.881781e-02,
        -3.771800e-05, -1.446006e-03]])

In [125]:
trainy[0]

array([0., 0., 0., 0., 1., 0.], dtype=float32)

- 0 WALKING
- 1 WALKING_UPSTAIRS
- 2 WALKING_DOWNSTAIRS
- 3 SITTING
- 4 STANDING
- 5 LAYING

# Analyse Data

There are three main signal types in the raw data:
- total acceleration
- body acceleration
- body gyroscope

Each has three axes of data. This means that there are a total of __nine variables for each time step__. 

Further, each serie sof data has been partitioned into overlapping windows of 2.65 seconds of data, or 128 time steps.

These windows of data correspond to the windows of engineered features (rows) in the previous section.

This means that one row of data has (128×9), or 1,152, elements

In [30]:
len(trainX)

7352

In [31]:
len(testX)

2947

In [15]:
len(trainX[0])

128

# Train Model

Define and train model:

In [126]:
# fit and evaluate a model
def train_model(trainX, trainy):
     
    # define parameters
    verbose = 1
    epochs = 25
    batch_size = 64
    n_outputs = 6 # number of classes    
    time_steps = 4
    rows = 1
    columns = 32
    channels = 9 #number of features
    samples = trainX.shape[0]
    
    # define model
    model = Sequential()
    model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(time_steps, rows, columns, channels)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # reshape data into subsequences (samples, time steps, rows, cols, channels)
    trainX = trainX.reshape((samples, time_steps, rows, columns, channels))
        
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)    

    return model

In [127]:
model = train_model(trainX, trainy)

Epoch 1/25
7352/7352 [==============================] - 16s 2ms/step - loss: 0.5881 - accuracy: 0.7693
Epoch 2/25
7352/7352 [==============================] - 15s 2ms/step - loss: 0.2070 - accuracy: 0.9177
Epoch 3/25
7352/7352 [==============================] - 15s 2ms/step - loss: 0.1464 - accuracy: 0.9410
Epoch 4/25
7352/7352 [==============================] - 15s 2ms/step - loss: 0.1398 - accuracy: 0.9416
Epoch 5/25
7352/7352 [==============================] - 14s 2ms/step - loss: 0.1182 - accuracy: 0.9498
Epoch 6/25
7352/7352 [==============================] - 14s 2ms/step - loss: 0.1258 - accuracy: 0.9486
Epoch 7/25
7352/7352 [==============================] - 14s 2ms/step - loss: 0.1075 - accuracy: 0.9539
Epoch 8/25
7352/7352 [==============================] - 14s 2ms/step - loss: 0.1105 - accuracy: 0.9538
Epoch 9/25
7352/7352 [==============================] - 15s 2ms/step - loss: 0.1041 - accuracy: 0.9554
Epoch 10/25
7352/7352 [==============================] - 14s 2ms/step - l

Try model

In [135]:
# prepare Data
samples = 1
time_steps = 4
rows = 1
columns = 32
channels = 9 #number of features

In [136]:
# reshape data into subsequences (samples, time steps, rows, cols, channels)
sample = trainX[:1].reshape((samples, time_steps, rows, columns, channels))

In [139]:
model.predict_proba(sample)

array([[4.0313435e-07, 4.7628955e-06, 1.9645756e-07, 3.4522134e-04,
        9.9964929e-01, 1.3913454e-07]], dtype=float32)

In [140]:
model.predict_classes(sample)

array([4], dtype=int64)

- 0 WALKING
- 1 WALKING_UPSTAIRS
- 2 WALKING_DOWNSTAIRS
- 3 SITTING
- 4 STANDING
- 5 LAYING

# Evaluate Performance

In [141]:
def evaluate_performance(testX, testy):

    # reshape data into subsequences (samples, time steps, rows, cols, channels)
    samples = testX.shape[0]
    time_steps = 4
    rows = 1
    columns = 32
    channels = 9 #number of features    
    testX = testX.reshape((samples, time_steps, rows, columns, channels))
    
    loss, accuracy = model.evaluate(testX, testy, verbose=0)
    return loss, accuracy   

In [142]:
loss, accuracy = evaluate_performance(testX, testy)

In [143]:
loss, accuracy

(0.6291025290808229, 0.9022734761238098)

# Store Performance on Hadoop

Create Performance Data

In [167]:
performance = {
    "timestamp":[datetime.now().strftime("%d/%m/%Y %H:%M:%S")],
    "accuracy":[accuracy]              
              }

In [168]:
performance

{'timestamp': ['27/01/2020 11:04:55'], 'accuracy': [0.9022734761238098]}

Write to Hadoop

In [169]:
path = "/tmp/tbr/BARMER/DSP/model/model_performance.csv"

with client_hdfs.write(path, encoding = 'utf-8', overwrite=True) as writer:
    pd.DataFrame(performance).to_csv(writer, mode='a', header=None, index=None)

Check results

In [170]:
client_hdfs.list('/tmp/tbr/BARMER/DSP/model/')

['model_performance.csv', 'model_structure.json', 'model_weights.h5']

# Store Model on Hadoop

## Save model structure

Serialize as JSON

In [171]:
model_json = model.to_json()
model_json[:100]

'{"class_name": "Sequential", "config": {"name": "sequential_2", "layers": [{"class_name": "ConvLSTM2'

Write to Hadoop

In [172]:
path = "/tmp/tbr/BARMER/DSP/model/model_structure.json"
with client_hdfs.write(path, encoding = 'utf-8', overwrite=True) as writer:
    writer.write(model_json)

Check result

In [173]:
client_hdfs.list('/tmp/tbr/BARMER/DSP/model/')

['model_performance.csv', 'model_structure.json', 'model_weights.h5']

## Save model weights

Serialize as local H5 file

In [174]:
# serialize weights to HDF5
model.save_weights("model_weights.h5")

Upload File to Hadoop

In [175]:
path = "/tmp/tbr/BARMER/DSP/model/model_weights.h5"
_ = client_hdfs.upload(hdfs_path=path, local_path="model_weights.h5", overwrite=True)

Check result

In [176]:
client_hdfs.list('/tmp/tbr/BARMER/DSP/model/')

['model_performance.csv', 'model_structure.json', 'model_weights.h5']